In [2]:
import ee
import geemap
import sys,os
sys.path.append(r'D:\09_Code\Gis_Script\GEE_Func')
from S1_distor_dedicated import load_S1collection,S1_CalDistor,DEM_caculator
from S2_filter import merge_s2_collection
from GEE_DataIOTrans import BandTrans,DataTrans,DataIO
from GEE_CorreterAndFilters import S1Corrector
import math
from functools import partial
from tqdm import tqdm 

# 重载函数
# import importlib
# importlib.reload(S1_distor_dedicated)

geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()

d:\Wrz\anaconda\envs\GEE\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# 获取S1 、 DEM
AOI =  ee.Geometry.Polygon([[[93.15172089782506, 30.60689894788499],
   [93.20966268713462, 30.60689894788499],
   [93.20966268713462, 30.63625137380944],
   [93.15172089782506, 30.63625137380944],
   [93.15172089782506, 30.60689894788499]]])

year = '2019'
START_DATE  = ee.Date(year + '-01-01')
END_DATE   = ee.Date(year + '-12-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
Origin_scale = 10
Prj_scale = 30


s1_ascending, s1_descending = load_S1collection(AOI,START_DATE,END_DATE,MIDDLE_DATE,FilterSize=30)
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()

S1_image = s1_ascending
DEM = DEMSRTM
Projection = S1_image.select(0).projection()
Mask = S1_image.select(0).mask()

# 获取辅助线
azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict  = S1Corrector.getS1Corners(s1_ascending, AOI,'ASCENDING')

Heading = azimuthEdge.get('azimuth')
s1_azimuth_across = ee.Number(Heading).subtract(90.0) # 距离向
Auxiliarylines = ee.Geometry.LineString([startpoint, endpoint])

Eq_pixels = DataTrans.Eq_pixels

BandTrans.delBands(S1_image, ['VH', 'VV'])
Cal_image = (Eq_pixels(BandTrans.delBands(S1_image, 'VV','VH').resample('bicubic')).rename('angle')
                      .addBands(ee.Image.pixelCoordinates(Projection))
                      .addBands(DEM.select('elevation'))
                      .addBands(ee.Image.pixelLonLat())
                      .updateMask(Mask)
                      .reproject(crs=Projection, scale=Prj_scale)
                      .clip(AOI))
Templist_A = S1_CalDistor.AuxiliaryLine2Point(Cal_image, s1_azimuth_across, coordinates_dict, Auxiliarylines, Origin_scale)

Without Filter


In [30]:
line_geometry = ee.Feature(Templist_A.get(3)).intersection(AOI).geometry()

# 获取线段的总长度
length = line_geometry.length()

# 获取线的所有坐标点
coordinates = line_geometry.coordinates()
start_point = ee.List(coordinates.get(0)) # 起点坐标
end_point = ee.List(coordinates.get(-1))  # 终点坐标

num_points = length.divide(30).subtract(1).floor()

In [32]:
num_points

In [5]:
import numpy as np
from scipy.optimize import least_squares

# 面转线-->获取4、9邻域-->计算代求点高程

def Line2Points(feature,scale=30):
    # 从Feature中提取线几何对象
    line_geometry = ee.Feature(feature).geometry()
    
    # 获取线的所有坐标点
    coordinates = line_geometry.coordinates()
    start_point = ee.List(coordinates.get(0)) # 起点坐标
    end_point = ee.List(coordinates.get(-1))  # 终点坐标
    
    # 获取线段的总长度
    length = line_geometry.length()
    num_points = length.divide(scale).subtract(1).floor()

    # 定义函数生成间隔点

    # 计算每个间隔点的坐标
    def interpolate(i):
        i = ee.Number(i)
        fraction = i.divide(num_points)
        interpolated_lon = ee.Number(start_point.get(0)).add(ee.Number(end_point.get(0)).subtract(ee.Number(start_point.get(0))).multiply(fraction))
        interpolated_lat = ee.Number(start_point.get(1)).add(ee.Number(end_point.get(1)).subtract(ee.Number(start_point.get(1))).multiply(fraction))
        return ee.Feature(ee.Geometry.Point([interpolated_lon, interpolated_lat]))
    
    # 使用map函数应用插值
    points = ee.FeatureCollection(ee.List.sequence(1, num_points).map(interpolate))
    return points

def get_neighborhood_info(point,image,scale=30,neighborhood_type='4'):
    
    point = ee.Geometry.Point(point)
    buffer_region = point.buffer(30)

    # 生成矩形区域
    region = buffer_region.bounds()
    # 获取矩形的坐标
    coords  = ee.List(region.coordinates().get(0))

    # 提取四个角点坐标
    corner1 = ee.Geometry.Point(coords.get(0))
    corner2 = ee.Geometry.Point(coords.get(1))
    corner3 = ee.Geometry.Point(coords.get(2))
    corner4 = ee.Geometry.Point(coords.get(3))
    
    if neighborhood_type=='4':
        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                                                corner1.coordinates(), corner2.coordinates(), 
                                                corner3.coordinates(), corner4.coordinates()])
    
    elif neighborhood_type=='9': 
        # 创建线段并计算边的中心点
        edge_center1 = ee.Geometry.LineString([corner1.coordinates(), corner2.coordinates()]).centroid()
        edge_center2 = ee.Geometry.LineString([corner2.coordinates(), corner3.coordinates()]).centroid()
        edge_center3 = ee.Geometry.LineString([corner3.coordinates(), corner4.coordinates()]).centroid()
        edge_center4 = ee.Geometry.LineString([corner4.coordinates(), corner1.coordinates()]).centroid()

        # 创建包含所有点的MultiPoint几何对象
        region_coord = ee.Geometry.MultiPoint([
                        corner1.coordinates(), corner2.coordinates(), 
                        corner3.coordinates(), corner4.coordinates(),
                        edge_center1.coordinates(), edge_center2.coordinates(), 
                        edge_center3.coordinates(), edge_center4.coordinates(),
                        point.coordinates()])
    else:
        print('neighborhood_type should be "4" or "9"')

    # 使用MultiPoint几何对象作为geometry参数
    result = image.reduceRegion(
        reducer=ee.Reducer.toList(),  # 转换为列表
        geometry=region_coord,
        scale=scale,
        maxPixels=1e9)

    # 在结果中添加点坐标
    result = result.set('point_coordinates', point.coordinates())
    
    return result 

def weighted_avg_elevation(neighbor):
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    neighbors_info = ee.Dictionary(neighbor)

    # 将邻域点信息转换为ee.Array
    x_coords = ee.Array(neighbors_info.get('longitude'))
    y_coords = ee.Array(neighbors_info.get('latitude'))
    elevations = ee.Array(neighbors_info.get('elevation'))
    point_coords  = ee.List(neighbors_info.get('point_coordinates'))
    # 计算距离和权重
    point1_x = ee.Number(point_coords.get(0))
    point1_y = ee.Number(point_coords.get(1))
    distances = x_coords.subtract(point1_x).pow(2).add(y_coords.subtract(point1_y).pow(2)).sqrt()
    weights = distances.pow(-1)

    # 计算加权平均高程
    weighted_elevations = elevations.multiply(weights)
    sum_weights = weights.reduce(ee.Reducer.sum(), [0])
    weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights)

    return weighted_avg_elevation

def avg_elevation(neighbor):
    neighbors_info = ee.Dictionary(neighbor)
    elevations = ee.Array(neighbors_info.get('elevation'))
    return elevations.reduce(ee.Reducer.mean(),[0])

def Volum_elavation(neighbors_info):
    
    # 修改方程函数以匹配新的模型
    def equation(params, x, y):
        a, b, c, d, e, f = params
        return a*x**2 + b*y**2 + c*x*y + d*x + e*y + f

    # 修改残差函数，去除z作为变量
    def residuals(params, x, y, z_true):
        z_pred = equation(params, x, y)
        return z_pred - z_true
    
    # 根据求得的参数和给定点的经纬度计算高程
    def calculate_z(params, x, y):
        # 根据新模型参数更新，只有6个参数
        a, b, c, d, e, f = params
        # 更新方程以反映新的模型形式
        z_pred = a*x**2 + b*y**2 + c*x*y + d*x + e*y + f
        return z_pred
    

    lon = np.array(neighbors_info['longitude'])
    lat = np.array(neighbors_info['latitude'])
    elevation = np.array(neighbors_info['elevation'])
    lon_pre, lat_pre = np.array(neighbors_info['point_coordinates'])

    # 初始参数猜测
    initial_guess = np.zeros(6)

    # 解最小二乘问题
    result = least_squares(residuals, initial_guess, args=(lon, lat, elevation))

    # 使用最小二乘法求得的参数
    params = result.x  

    # 计算给定点的预测高程z_pred
    z_pred = calculate_z(params, lon_pre, lat_pre)

    return z_pred

def Area_elavation(neighbors_info):


    lon = neighbors_info['longitude']
    lat = neighbors_info['latitude']
    elevation = neighbors_info['elevation']
    x_pred, y_pred = np.array(neighbors_info['point_coordinates'])
    
    A = np.vstack([lon, lat, np.ones(len(lon))]).T  # 构建矩阵A
    b = elevation  # 高程向量b

    # 使用最小二乘法解方程Ax = b
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

    # x包含拟合参数a, b, c（注意这里不再是1/c）
    a, b, c = x

    z_pred = a * x_pred + b * y_pred + c
    return z_pred

def apply_map_to_list(input_list, func): return input_list.map(func)
def apply_map_to_list_local(input_list, func): return map(func,input_list)


# 使用一条线做测试
from tqdm import tqdm

Len_Templist_A =  Templist_A.size().getInfo()
pbar = tqdm(range(Len_Templist_A),ncols=80)
Neighbors = '4' # '9'

# 需要注意'Area_elavation','Volum_elavation'涉及数学拟合，GEE-->Python环境
Elvevation_model = 'weighted_avg_elevation' # 'avg_elevation', 'Area_elavation','Volum_elavation'

test_list = [] 
for i in pbar:
    points = Line2Points(Templist_A.get(i),scale=Prj_scale)  
    points = points.geometry().coordinates()
    test_list.append(points)
    
test_list = ee.List(test_list[0:2])

cal_neighbors = apply_map_to_list(test_list, 
                               lambda x: ee.List(x).map(partial(get_neighborhood_info,image=Cal_image,scale=Prj_scale,neighborhood_type=Neighbors))
                               )

if Elvevation_model == 'weighted_avg_elevation':
    points_elevations_neighborhood = apply_map_to_list(
                                        cal_neighbors,      
                                        lambda x: ee.List(x).map(weighted_avg_elevation)
                                                        )
elif Elvevation_model == 'avg_elevation':
    points_elevations_avg = apply_map_to_list(
                                        cal_neighbors,
                                        lambda x: ee.List(x).map(avg_elevation))
elif (Elvevation_model == 'Area_elavation') or (Elvevation_model == 'Volum_elavation'):
    neighbors_info = cal_neighbors.getInfo()
    
    if Elvevation_model == 'Area_elavation':
        points_elevations_area = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Area_elavation,x))))
    elif Elvevation_model == 'Volum_elavation':
        points_elevations_volum = list(apply_map_to_list_local(neighbors_info,lambda x: list(map(Volum_elavation,x))))

100%|████████████████████████████████████████| 557/557 [00:05<00:00, 109.26it/s]


In [6]:
# 滤波点。 根据高程和距离？ ，检测高差？

In [7]:
# 初始化地图
Map = geemap.Map()

# 定义两个存在交集的多边形
polygon1 = ee.Geometry.Polygon([
    [[-122.092, 37.424],
     [-122.086, 37.418],
     [-122.079, 37.425],
     [-122.085, 37.423]]
])

polygon2 = ee.Geometry.Polygon([
    [[-122.085, 37.423],
     [-122.078, 37.416],
     [-122.070, 37.423],
     [-122.077, 37.426]]
])

# 求两个多边形的交集
intersection = polygon1.intersection(polygon2)

# 将多边形和它们的交集添加到地图上
Map.addLayer(polygon1, {'color': 'blue'}, 'Polygon 1')
Map.addLayer(polygon2, {'color': 'green'}, 'Polygon 2')
Map.addLayer(intersection, {'color': 'red'}, 'Intersection')

# 设置地图的中心点和缩放级别
Map.setCenter(-122.085, 37.422, 15)

# 显示地图
Map

Map(center=[37.422, -122.085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…